In [2]:
import datetime
import json
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit_experiments.framework import ExperimentEncoder
from qiskit_research.mzm_generation import (
    KitaevHamiltonianExperiment,
    KitaevHamiltonianExperimentParameters,
)
from qiskit_research.mzm_generation.utils import transpile_circuit
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations

from qiskit.circuit.library import RZGate
from qiskit_research.utils.dynamical_decoupling import get_instruction_durations

from qiskit.visualization import timeline_drawer

In [3]:
# get backend

# uncomment to use hardware backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend_name = 'ibmq_guadalupe'

# Aer simulator backend
# provider = None
# backend_name = "aer_simulator"

In [6]:
# experiment parameters
n_modes = 6
tunneling_values = [-1.0]
superconducting_values = [1.0]
# chemical_potential_values = list(np.linspace(0.0, 3.0, num=5))
chemical_potential_values = [3]
occupied_orbitals_list = list(orbital_combinations(n_modes, threshold=2))
# dynamical_decoupling_sequences = None
dynamical_decoupling_sequences=[None]
num_dd_passes=1
uhrig_spacing = False
concat_layers = 1
shots = 10000
readout_calibration_shots = 10000

# pick qubits
qubits, backend_name, error_score = pick_qubit_layout(n_modes, backend_name, provider)
print(f"Using qubits {qubits}.")

# date
date = datetime.datetime.now().isoformat()

# create experiment
params = KitaevHamiltonianExperimentParameters(
    timestamp=date,
    backend_name=backend_name,
    qubits=qubits,
    n_modes=n_modes,
    tunneling_values=tunneling_values,
    superconducting_values=superconducting_values,
    chemical_potential_values=chemical_potential_values,
    occupied_orbitals_list=occupied_orbitals_list,
    dynamical_decoupling_sequences=dynamical_decoupling_sequences,
    num_dd_passes=num_dd_passes,
    uhrig_spacing=uhrig_spacing,
    concat_layers=concat_layers,
    seed=11364,
)
experiment = KitaevHamiltonianExperiment(params, provider)

print(f"Total number of circuits: {len(experiment.circuits())}")

Using qubits [12, 13, 14, 11, 8, 5].
Total number of circuits: 78


In [ ]:
circ = transpile_circuit(
    experiment.circuits()[0],
    experiment.backend,
    initial_layout=list(experiment.physical_qubits),
    dynamical_decoupling_sequence=dynamical_decoupling_sequences[0],
    num_dd_passes=1,
    uhrig_spacing=False,
    concat_layers=1,
    pulse_scaling=experiment.params.pulse_scaling,
    pauli_twirling=bool(experiment.params.num_twirled_circuits),
    seed=experiment.rng,
)
timeline_drawer(circ).savefig('foldertest/test.png')

In [4]:
sequences = ['X2pm', 'XY4pm', 'XY8pm']
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = [3]

for n in range(12, 13):
    occupied_orbitals_list = list(orbital_combinations(n, threshold=2))
    qubits, backend_name, error_score = pick_qubit_layout(n, backend_name, provider)
    date = datetime.datetime.now().isoformat()
    
    for sequence in sequences:
        folder_name = f"timeline-diagrams/n{n}/{sequence}"
        
        params = KitaevHamiltonianExperimentParameters(
            timestamp=date,
            backend_name=backend_name,
            qubits=qubits,
            n_modes=n,
            tunneling_values=tunneling_values,
            superconducting_values=superconducting_values,
            chemical_potential_values=chemical_potential_values,
            occupied_orbitals_list=occupied_orbitals_list,
            dynamical_decoupling_sequences=[sequence],
            num_dd_passes=1,
            uhrig_spacing=False,
            concat_layers=1,
            seed=11364,
        )

        experiment = KitaevHamiltonianExperiment(params, provider)

        i = 1
        for circuit in experiment.circuits():
            circ = transpile_circuit(
                circuit,
                experiment.backend,
                initial_layout=list(experiment.physical_qubits),
                dynamical_decoupling_sequence=sequence,
                num_dd_passes=1,
                uhrig_spacing=False,
                concat_layers=1,
                pulse_scaling=experiment.params.pulse_scaling,
                pauli_twirling=bool(experiment.params.num_twirled_circuits),
                seed=experiment.rng,
            )

            os.makedirs(os.path.dirname(f"{folder_name}/{i}.png"), exist_ok=True)
            timeline_drawer(circ).savefig(f"{folder_name}/{i}.png")
            i += 1
            